## 建立机器学习管道

[Azure Machie Learning Pipeline](https://docs.microsoft.com/zh-cn/azure/machine-learning/concept-ml-pipelines) 实现可重用的机器学习管道。
它可以用于模型学习和批处理推断。内置的机器学习管道是[Github Actions](https://github.com/Azure/aml-run) 或者 [Azure Data Factory](https://docs.microsoft.com/zh-cn/azure/data-factory/transform-data-machine-learning-service) 可以从诸如之类的服务中调用它。


在此Noteobok中，将[3-aml-run.ipynb](./3-aml-run.ipynb)的处理重构为管道。

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Dataset,Environment
from azureml.widgets import RunDetails

### 机器学习管道相关的库

In [ ]:
from azureml.pipeline.core import Pipeline, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
from azureml.core.compute import AmlCompute
aml_compute = AmlCompute(ws, "compute1")

### 执行配置

In [ ]:
from azureml.core.runconfig import RunConfiguration
run_config = RunConfiguration()
run_config.environment.docker.enabled = True
env = Environment.get(ws, "pytorch-env")
run_config.environment = env

### 数据集

In [ ]:
dataset = Dataset.get_by_name(ws, name='cifar10')
ds_input = dataset.as_named_input('input1')

### `PythonScriptStep`执行配置

执行python脚本的模块的执行配置

In [ ]:
step1 = PythonScriptStep(name="train_step",
                         script_name="train.py",
                         arguments=[
                            '--data_path', ds_input.as_download(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92],
                         #inputs=[ds_input],
                         compute_target=aml_compute, 
                         source_directory='./code/pytorch-cloud',
                         runconfig=run_config,
                         allow_reuse=True)
print("step1 创建完成")

In [ ]:
steps = [step1]

In [ ]:
pipeline1 = Pipeline(workspace=ws, steps=steps)
print ("Pipeline 构筑完成")

In [ ]:
pipeline1.validate()
print ("Pipeline 验证完成")

In [ ]:
run = pipeline1.submit("5-aml-pipeline-run")

In [ ]:
run.wait_for_completion()

### 发布为终结点

In [ ]:
pipeline1.publish("5-aml-pipeline")